In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
# Создание датафрейма с dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
    common_feat = list( set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [3]:
features = ['Внешность', 'Алкоголь_в_напитке',
            'Уровень_красноречия', 'Потраченные_деньги']
df_train = {}
df_train['Внешность'] = ['приятная', 'приятная', 'приятная', 'отталкивающая',
                         'отталкивающая', 'отталкивающая', 'приятная'] 
df_train['Алкоголь_в_напитке'] = ['да', 'да', 'нет', 'нет', 'да', 'да', 'да']
df_train['Уровень_красноречия'] = ['высокий', 'низкий', 'средний', 'средний', 'низкий',
                                   'высокий', 'средний']
df_train['Потраченные_деньги'] = ['много', 'мало', 'много', 'мало', 'много',
                                  'много', 'много']
df_train['Поедет'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

,Поедет,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_низкий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,0,1,1,0,1,0,0,0,1
1,1,0,1,1,0,0,1,0,1,0
2,0,0,1,0,1,0,0,1,0,1
3,1,1,0,0,1,0,0,1,1,0
4,1,1,0,1,0,0,1,0,0,1
5,0,1,0,1,0,1,0,0,0,1
6,0,0,1,1,0,0,0,1,0,1


In [4]:
df_test = {}
df_test['Внешность'] = ['приятная', 'приятная', 'отталкивающая'] 
df_test['Алкоголь_в_напитке'] = ['нет', 'да', 'да']
df_test['Уровень_красноречия'] = ['средний', 'высокий', 'средний']
df_test['Потраченные_деньги'] = ['много', 'мало', 'много']
df_test = create_df(df_test, features)
df_test

,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,1,0,1,0,1,0,1
1,0,1,1,0,1,0,1,0
2,1,0,1,0,0,1,0,1


In [5]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y = df_train['Поедет']
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

,Внешность_приятная,Алкоголь_в_напитке_нет,Внешность_отталкивающая,Уровень_красноречия_средний,Алкоголь_в_напитке_да,Потраченные_деньги_много,Потраченные_деньги_мало,Уровень_красноречия_высокий
0,1,0,0,0,1,1,0,1
1,1,0,0,0,1,0,1,0
2,1,1,0,1,0,1,0,0
3,0,1,1,1,0,0,1,0
4,0,0,1,0,1,1,0,0
5,0,0,1,0,1,1,0,1
6,1,0,0,1,1,1,0,0


In [6]:
df_test

,Внешность_приятная,Алкоголь_в_напитке_нет,Внешность_отталкивающая,Уровень_красноречия_средний,Алкоголь_в_напитке_да,Потраченные_деньги_много,Потраченные_деньги_мало,Уровень_красноречия_высокий
0,1,1,0,1,0,1,0,0
1,1,0,0,0,1,0,1,1
2,0,0,1,1,1,1,0,0


In [7]:
balls_left  = [1 for i in range(8)] + [0 for i in range(5)] # 8 синих и 5 желтых
balls_right = [1 for i in range(1)] + [0 for i in range(6)] # 1 синий и 6 желтых

In [12]:
def entropy(a_list):
    z_count = 0
    o_count = 0
    for value in a_list:
        if value == 1:
            o_count = o_count + 1
        else:
            z_count = z_count + 1
    count = z_count + o_count
    if count > 0:
        return -(z_count/count) * np.log2(z_count/count) - (o_count/count) * np.log2(o_count/count)
    else:
        return 0

In [13]:
entropy(balls_left)

0.9612366047228759

In [14]:
entropy(balls_right)

0.5916727785823275

In [15]:
6*(-1/6)*np.log2(1/6)

2.584962500721156

In [16]:
entropy(balls_left + balls_right)

0.9927744539878083

In [17]:
0.992774 - (13/20)*0.9612366 - (7/20)*0.591672778582

0.1608847374963001

In [32]:
data_train = pd.read_csv('adult.data', sep=' ') 

In [19]:
df_test = pd.read_csv('adult_train.csv')

In [31]:
data_train.T.tail()

,0,1,2,3,4,5,6,7,8,9,...,32550,32551,32552,32553,32554,32555,32556,32557,32558,32559
"2174,","0,","0,","0,","0,","0,","0,","0,","14084,","5178,","0,",...,"0,","0,","0,","0,","0,","0,","0,","0,","0,","15024,"
"0,","0,","0,","0,","0,","0,","0,","0,","0,","0,","0,",...,"0,","0,","0,","0,","0,","0,","0,","0,","0,","0,"
"40,","13,","40,","40,","40,","40,","16,","45,","50,","40,","80,",...,"40,","45,","11,","40,","40,","38,","40,","40,","20,","40,"
"United-States,","United-States,","United-States,","United-States,","Cuba,","United-States,","Jamaica,","United-States,","United-States,","United-States,","United-States,",...,"United-States,","United-States,","Taiwan,","United-States,","United-States,","United-States,","United-States,","United-States,","United-States,","United-States,"
<=50K,<=50K,<=50K,<=50K,<=50K,<=50K,<=50K,>50K,>50K,>50K,>50K,...,<=50K,<=50K,<=50K,>50K,<=50K,<=50K,>50K,<=50K,<=50K,>50K


In [21]:
df_test.tail()

,Age;Workclass;fnlwgt;Education;Education_Num;Martial_Status;Occupation;Relationship;Race;Sex;Capital_Gain;Capital_Loss;Hours_per_week;Country;Target
32556,27; Private;257302; Assoc-acdm;12; Married-civ...
32557,40; Private;154374; HS-grad;9; Married-civ-spo...
32558,58; Private;151910; HS-grad;9; Widowed; Adm-cl...
32559,22; Private;201490; HS-grad;9; Never-married; ...
32560,52; Self-emp-inc;287927; HS-grad;9; Married-ci...


In [22]:
# необходимо убрать строки с неправильными метками в тестовой выборке
data_test = df_test[(df_test['Target'] == ' >50K.') 
                      | (df_test['Target']==' <=50K.')]

data_train = df
# перекодируем target в числовое поле
data_train.at[data_train['Target'] == ' <=50K', 'Target'] = 0
data_train.at[data_train['Target'] == ' >50K', 'Target'] = 1

data_test.at[data_test['Target'] == ' <=50K.', 'Target'] = 0
data_test.at[data_test['Target'] == ' >50K.', 'Target'] = 1

KeyError: 'Target'